# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system

In [3]:
A = rand(3, 3);
x = fill(1, (3,));
b = A * x;
print(A)
print(x)
print(b)

[0.880816 0.84889 0.803494; 0.735591 0.644518 0.435516; 0.0277079 0.234894 0.189904][1, 1, 1][2.5332, 1.81562, 0.452506]

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA=LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [4]:
Alu = lufact(A)

Base.LinAlg.LU{Float64,Array{Float64,2}} with factors L and U:
[1.0 0.0 0.0; 0.0314571 1.0 0.0; 0.835125 -0.309386 1.0]
[0.880816 0.84889 0.803494; 0.0 0.208191 0.164628; 0.0 0.0 -0.184568]

In [5]:
typeof(Alu)

Base.LinAlg.LU{Float64,Array{Float64,2}}

The different parts of the factorization can be extracted by special indexing

In [6]:
Alu[:P]

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0

In [7]:
Alu[:L]

3×3 Array{Float64,2}:
 1.0         0.0       0.0
 0.0314571   1.0       0.0
 0.835125   -0.309386  1.0

In [8]:
Alu[:U]

3×3 Array{Float64,2}:
 0.880816  0.84889    0.803494
 0.0       0.208191   0.164628
 0.0       0.0       -0.184568

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [9]:
A\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

In [10]:
Alu\b

3-element Array{Float64,1}:
 1.0
 1.0
 1.0

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [11]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [12]:
Aqr = qrfact(A)

Base.LinAlg.QRCompactWY{Float64,Array{Float64,2}} with factors Q and R:
[-0.767321 0.12944 -0.628064; -0.640809 -0.191641 0.743396; -0.0241377 0.972892 0.229997]
[-1.14791 -1.07005 -0.900204; 0.0 0.214891 0.205297; 0.0 0.0 -0.137207]

The matrices `Q` and `R` can be extracted from the QR factorization object via

In [13]:
Aqr[:Q]

3×3 Base.LinAlg.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.767321    0.12944   -0.628064
 -0.640809   -0.191641   0.743396
 -0.0241377   0.972892   0.229997

In [14]:
Aqr[:R]

3×3 Array{Float64,2}:
 -1.14791  -1.07005   -0.900204
  0.0       0.214891   0.205297
  0.0       0.0       -0.137207

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [15]:
Asym = A + A'
AsymEig = eigfact(Asym)

Base.LinAlg.Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}([-0.0874352, 0.0246577, 3.49325], [0.694686 -0.0323206 -0.718587; -0.656653 -0.436292 -0.615188; -0.293631 0.899224 -0.324309])

The values and the vectors can be extracted from the Eigen type by special indexing

In [16]:
AsymEig[:values]

3-element Array{Float64,1}:
 -0.0874352
  0.0246577
  3.49325  

In [17]:
AsymEig[:vectors]

3×3 Array{Float64,2}:
  0.694686  -0.0323206  -0.718587
 -0.656653  -0.436292   -0.615188
 -0.293631   0.899224   -0.324309

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [18]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
  1.0          6.66134e-16   2.27596e-15
 -1.42109e-14  1.0          -5.32907e-15
  2.4869e-14   2.84217e-14   1.0        

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [19]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [20]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [21]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.44296557646288703

In [22]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [23]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [24]:
@time eigvals(Asym);

  0.337375 seconds (18.08 k allocations: 8.840 MiB, 1.51% gc time)


In [25]:
@time eigvals(Asym_noisy);

  1.190157 seconds (24 allocations: 7.921 MiB)


In [26]:
@time eigvals(Asym_explicit);

  0.383990 seconds (1.86 k allocations: 8.088 MiB, 2.14% gc time)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [27]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.889573 seconds (77.35 k allocations: 187.366 MiB, 16.27% gc time)


6.913858964725685

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [28]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [29]:
Arational = convert(Matrix{Rational{BigInt}}, rand(1:10,3,3))/10

3×3 Array{Rational{BigInt},2}:
 1//10  7//10  7//10
 3//10  3//5   1//2 
 3//5   7//10  9//10

In [30]:
x = fill(1, (3))
b = Arational*x

3-element Array{Rational{BigInt},1}:
  3//2
  7//5
 11//5

In [31]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [32]:
lufact(Arational)

Base.LinAlg.LU{Rational{BigInt},Array{Rational{BigInt},2}} with factors L and U:
Rational{BigInt}[1//1 0//1 0//1; 3//1 1//1 0//1; 6//1 7//3 1//1]
Rational{BigInt}[1//10 7//10 7//10; 0//1 -3//2 -8//5; 0//1 0//1 13//30]

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

In [ ]:
sol = eigfact(A)

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

#### 11.3 
Create a `LowerTriangular` matrix from `A`.

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback